We now use a wave speed that is discontinuous at $x=0$. 

In [ ]:
include("../FNC.jl")
m = 120
x,Dx = FNC.diffcheb(m,[-1,1]);
c = @. 1 + (sign(x)+1)/2
chop(u) = u[2:m];
extend(v) = [0;v;0];

This function computes the time derivative of the method-of-lines system.

In [ ]:
dwdt = function (w,c,t)
    u = extend(w[1:m-1])
    z = w[m:2*m]
    dudt = Dx*z
    dzdt = c.^2 .* (Dx*u)
    return [ chop(dudt); dzdt ]
    end;

We set the initial conditions and solve using |ode45|.

In [ ]:
u_init = @.exp(-100*(x+0.5)^2);
z_init = -u_init;
w_init = [ chop(u_init); z_init ];    
using DifferentialEquations
IVP = ODEProblem(dwdt,w_init,(0.,2.),c)
sol = solve(IVP,RK4());

Now we extract the original $u$ and $z$ variables from the results, adding in the zeros at the boundaries for $u$.

In [ ]:
n = length(t)-1;
U = [ zeros(1,n+1); W(1:m-1,:); zeros(1,n+1) ];
Z = W(m:2*m,:);

Finally, we plot the results.

In [ ]:
using Plots
@gif for t = 2*(0:100)/100 
    plot(x,extend(sol(t,idxs=1:m-1)),
        xaxis=("x"),yaxis=([-1,1],"u(x,t)"),    
        title="Wave equation, variable speed",label="t=$(round(t,digits=3))" )
end

Each pass through the interface at $x=0$ generates a reflected and transmitted wave. By conservation of energy, these are both smaller in amplitude than the incoming bump. 